In [2]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
# from pandas import datetime
from pmdarima.arima import auto_arima
import statsmodels.api as sm
import warnings

In [3]:
path = './open'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [4]:
start_date = '20210104'
end_date = '20211031'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 43
HOW MANY "Business_days" : (215, 1)


,Date
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08


In [5]:
sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
sample = pd.merge(Business_days, sample, how = 'outer')
sample['weekday'] = sample.Date.apply(lambda x : x.weekday())
sample['weeknum'] = sample.Date.apply(lambda x : x.strftime('%V'))
sample.Close = sample.Close.ffill()
sample = pd.pivot_table(data = sample, values = 'Close', columns = 'weekday', index = 'weeknum')

sample_flatten = np.ravel(sample.to_numpy())
len(sample_flatten)



215

In [6]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = len(X)-5
	train, test = X[0:train_size], X[train_size:]

	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(train, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast(steps=5)
		predictions.append(yhat)
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error

In [7]:

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))

	return

In [ ]:
p_values=[0,1,2,3]
d_values=[1,2]
q_values=[0,1,2,3]
warnings.filterwarnings("ignore")

evaluate_models(sample_flatten,p_values,d_values,q_values)    

[70200. 71100. 70100. 70700. 69800.]
[69874.00276429 69773.77802272 69851.21729171 69737.04186444
 69740.93102896]


In [8]:
sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,sample_name))
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-11-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-11-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-11-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-11-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2021-11-29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2021-11-30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2021-12-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2021-12-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2021-12-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:

for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    
    data_flatten = np.ravel(data.to_numpy())
    
    model=ARIMA(data_flatten,order=(0,2,1))
    model_fit=model.fit()

    predictions=model_fit.forecast(steps=5)
    print(predictions)
    # break
    sample_submission.loc[0:4,code] = predictions
sample_submission.isna().sum().sum()

  0%|          | 1/370 [00:00<02:12,  2.79it/s]

[68952.16670852 68104.33341704 67256.50012556 66408.66683409
 65560.83354261]


  1%|          | 2/370 [00:00<02:20,  2.63it/s]

[100797.35853299  98594.71706598  96392.07559897  94189.43413195
  91986.79266494]


  1%|          | 3/370 [00:01<02:07,  2.88it/s]

...

 32%|███▏      | 118/370 [00:41<01:14,  3.40it/s]

[244973.74302568 241947.48605135 238921.22907703 235894.9721027
 232868.71512838]


 32%|███▏      | 119/370 [00:41<01:27,  2.86it/s]

[22260.91345504 22021.82691007 21782.74036511 21543.65382015
 21304.56727519]


KeyboardInterrupt: 

In [ ]:
sample_submission.columns

Index(['Day', '000060', '000080', '000100', '000120', '000150', '000240',
       '000250', '000270', '000660',
       ...
       '330860', '336260', '336370', '347860', '348150', '348210', '352820',
       '357780', '363280', '950130'],
      dtype='object', length=371)

In [ ]:
columns = list(sample_submission.columns[1:])

columns = ['Day'] + [str(x).zfill(6) for x in columns]

sample_submission.columns = columns

In [ ]:
sample_submission.to_csv('BASELINE_Linear.csv',index=False)

In [ ]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,27778.290498,34287.329892,59005.366550,138496.464702,103234.899188,16683.644556,45469.412046,85052.136063,100797.358533,...,49491.403714,52682.869338,82828.251387,35350.993921,25600.784437,52225.585465,335819.322716,259632.806738,27015.742529,17086.094982
1,2021-11-02,27856.580996,33774.659785,58110.733099,134992.929404,103469.798377,16667.289112,43888.824092,85004.272125,98594.717066,...,49882.807427,53165.738677,83356.502774,34601.987842,25251.568874,51351.170930,336638.645431,257665.613475,26881.485059,17072.189963
2,2021-11-03,27934.871495,33261.989677,57216.099649,131489.394106,103704.697565,16650.933668,42308.236138,84956.408188,96392.075599,...,50274.211141,53648.608015,83884.754161,33852.981763,24902.353311,50476.756395,337457.968147,255698.420213,26747.227588,17058.284945
3,2021-11-04,28013.161993,32749.319570,56321.466198,127985.858808,103939.596753,16634.578224,40727.648185,84908.544250,94189.434132,...,50665.614855,54131.477353,84413.005547,33103.975684,24553.137748,49602.341860,338277.290862,253731.226951,26612.970118,17044.379927
4,2021-11-05,28091.452491,32236.649462,55426.832748,124482.323510,104174.495941,16618.222780,39147.060231,84860.680313,91986.792665,...,51057.018568,54614.346691,84941.256934,32354.969605,24203.922185,48727.927324,339096.613578,251764.033689,26478.712647,17030.474908
5,2021-11-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,2021-11-30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,2021-12-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,2021-12-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,2021-12-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:

start_date = '20211101'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )


real=[]
real_data=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\stock_predict2\\open\\sample_submission.csv')
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()

    real_data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    real_data_close_flatten = np.ravel(real_data_close.to_numpy())
    # print(real_data_close_flatten)
    

    real_data.loc[:4,code]=real_data_close_flatten


type(real_data)
type(sample_submission)

label_np = real_data.to_numpy()[0:5,1:]
pred_np = sample_submission.to_numpy()[0:5, 1:]
print(pred_np)


result=np.mean(abs(label_np-pred_np)/label_np)*100
print(result)

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (5, 1)


100%|██████████| 370/370 [01:27<00:00,  4.21it/s]

[[27778.290498189148 34287.32989242792 59005.36654961816 ...
  259632.80673770854 27015.742529463645 17086.094981690447]
 [27856.580996378296 33774.659784855845 58110.73309923631 ...
  257665.61347541705 26881.48505892729 17072.189963380893]
 [27934.871494567444 33261.98967728377 57216.09964885448 ...
  255698.42021312556 26747.227588390935 17058.28494507134]
 [28013.16199275659 32749.319569711693 56321.46619847264 ...
  253731.22695083407 26612.97011785458 17044.379926761787]
 [28091.45249094574 32236.64946213962 55426.832748090805 ...
  251764.03368854258 26478.712647318225 17030.474908452234]]
4.882870265112119
